# data source 
https://pci.gov.ng/conversion-centers

In [42]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
from geopy.geocoders import Nominatim # for geocoding 
import re
import httpx

### Data Scrapping 

In [18]:
url = 'https://pci.gov.ng/conversion-centers'

response = requests.get(url)
soup = BeautifulSoup(response.content,"html.parser")

# check for the tag that holds all the information we want to extract
results = soup.find(class_="table table-striped")

# select all the table rows
table = results.find_all('tr')
link = results.find_all('a')

# now let extract etable data in the rows and store in in a list
data = []

for row in table[1:]: # skip the data , it contains empty data
    cols = row.find_all('td')
    rows = [info.text.strip() for info in cols]
    data.append(rows)

google_url = []
for url in link:
    google_url.append(url.get('href'))

# convert the list of data dataframe
# create the list of columns header
cols_header = ['State','LGA','Name','Address']
df = pd.DataFrame(columns=cols_header,data=data)

# drop all none fields
df.dropna(inplace=True)

# drop duplicates field
df.drop_duplicates(inplace=True)

# add the list url limk to the dataframe
df['google_map_url'] = google_url

# run a test to cofirm the total data scrapped is equal to 160, because according to the information on the website we havve 160 CNG stations
assert len(df) == 160, "The total Number of Data Scrapped is not equal to 160"


In [48]:
df

,State,LGA,Name,Address,google_map_url
0,FCT,Abuja Municipal Area Council,Nigeria Institute of Transport Technology (NIT...,"Plot 487, Asuquo Okon Street, Daki-Biyu, Jabi ...",https://maps.google.com/?q=Plot+487%2C+Asuquo+...
1,Lagos,Ibeju-Lekki,Femadec Energy Limited Lagos,"KM 42, Lekki-Epe Expressway, Majek Second Gate...",https://maps.google.com/?q=KM+42%2C+Lekki-Epe+...
2,Kaduna,Kaduna North,Rolling Energy Limited Kaduna State,"Plot 16715, Kaduna Abuja Expressway, Kakau Tow...",https://maps.google.com/?q=Plot+16715%2C+Kadun...
3,FCT,Abuja Municipal Area Council,ABG Oil and Gas Limited Abuja,"49FJ+G6R Kubwa 901101, Federal Capital Territo...",https://maps.google.com/?q=49FJ%2BG6R+Kubwa+90...
4,Kaduna,Zaria,Nigerian Institute of Transport Technology (NI...,"Basawa Road, Palladan, Zaria, Kaduna Conversio...",https://maps.google.com/?q=Basawa+Road%2C+Pall...
...,...,...,...,...,...
155,Delta,Uvwie,Dandani Gas Limited,"135 Refinery Road, KM3, Opposite Navy Air Base...",https://maps.google.com/?q=135+Refinery+Road%2...
156,Delta,Aniocha South,Negus Energy and Engineering Services Limited,"18, Old Site Mechanic Village, Asaba, Delta State",https://maps.google.com/?q=18%2C+Old+Site+Mech...
157,Kaduna,Kaduna North,Attayseer Autogas Nig Ltd,"NO 1, Gulf Club Cross Road, Al-Mansur Motors M...",https://maps.google.com/?q=NO+1%2C+Gulf+Club+C...
158,FCT,Municipal Area Council,Hi-Grade Energies Limited,Arish Mall Zuba Opposite Enyo Filling Station ...,https://maps.google.com/?q=Arish+Mall+Zuba+Opp...


### Geocoding


Convert the Address to longitude and latitude

#### Extraction using Geopy Objects for geocoding such as Nominatim and ArcGIS

In [20]:
# lon = []
# lat = []
# for _,col in df.iterrows():
#     geolocator = Nominatim(user_agent="samuel")
#     print(col['Name'])
#     location = geolocator.geocode(col['Name'])
#     print(location.longitude)
#     lon.append(location.longitude)
#     lat.append(location.latitude)

# lon,lat
# # df['longitude','latitude'] = 

#### Extract the lon and lat of the center view of the google map link addeded to the data

In [44]:
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/105.0.3729.169 Safari/537.36",
    "Referer": "https://www.google.com/"
}


In [ ]:
google_lon = []
google_lat = []

for _,col in df.iterrows():
    url = httpx.get(col['google_map_url'],headers=header)
    page_content = url.content.decode('utf-8').strip()  # Decode bytes to string and strip any extra spaces

    # Regular expression to extract center coordinates
    match = re.search(r'center=([\d\.\-]+)%2C([\d\.\-]+)', page_content)

    if match:
        latitude = match.group(1)
        longitude = match.group(2)

        # append the lon and lat to the empty list
        google_lon.append(longitude)
        google_lat.append(latitude)
    else:
        print(col['google_map_url'])
        print("No coordinates found.")


df['longitude'] = google_lon
df['latitude'] = google_lat

### Join External Datatst that has the localtions Long and Lat Dataset Generated from Using Google Sheet Google API Service